# HW2

Короткова Юлия

## Чтение данных

Данные я использовала "Musical_Instruments_5.json.gz" - датасет с отзывами о музыкальных инструментах.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [60]:
import pandas as pd
import gzip
import json
from collections import Counter, defaultdict
import numpy as np

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
stopwords = stopwords.words('english')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [4]:
df = getDF('/content/drive/MyDrive/Musical_Instruments_5.json.gz')

In [5]:
df = df[["asin", "reviewText"]]
df = df.dropna()

### 1.

1. Посчитать Tf-idf, где документ = коллекция отзывов для одного айтема. Таким образом с помощью idf вектора получится найти наиболее уникальные слова. А если какие-то слова уникальны для отзывов конкретного айтема, то мое предположение, что эти слова как-то характеризуют данный айтем. Если на такой топ наложить фильтр, например, на существительные, то это у будут искомые NE. То же можно сделать с биграммами, триграммами. Из данных нужны только отзывы сгруппированные по товарам. Из достоинств минимум ручной работы, из недостатков неточность, потому что будет много сущностей, которые не являются NE.

2. Так же поделить отзывы по айтемам и найти наиболее частотные для каждого конкретного айтема. Далее так же наложить фильтр и тоже получатся NE. Данные, достоинства и недостатки те же, что и в предыдущем подходе.

3. Если были бы метаданные (мне их так и не прислали), то из заголовков/описаний можно вытащить существительные (или какой-то другой фильтр, например, сущ + число для NE типа "Iphone 10"). Потом построить эмбеддинги по этим сущностям и по словам из отзывов и среди последних искать наиболее близкие к первым. В итоге тоже получается коллекция NE. В качестве данных еще нужны метаданные. По сравнению с предыдущими подходами точность должна повыситься, так как мы теперь ищем сущности более явные, но при этом и ручная работа добавляется.

## 2.

Я сначала хотела реализовать tf-idf, но этот способ не сработал, но я все равно хотела его показать, поэтому сначала приведен код для этого случая, а потом для другого метода с CountVectorzer.

### Предобработка данных

In [8]:
df_agg = df.groupby("asin").agg(list)
df_agg["texts"] = df_agg["reviewText"].apply(lambda x: " ".join(x))

In [44]:
df_agg

,reviewText,texts
asin,,
0739079891,"[It's good for beginners, I recommend this sta...",It's good for beginners I recommend this start...
0786615206,"[Very ordinary suctions, but strong. Can adjus...","Very ordinary suctions, but strong. Can adjust..."
1480360295,[So far it works as designed but it needs ridg...,So far it works as designed but it needs ridge...
1928571018,[This books proves to be handy to any self-lea...,This books proves to be handy to any self-lear...
9792372326,[I have been using QSC K10s for close to a cou...,I have been using QSC K10s for close to a coup...
...,...,...
B01HHJXKRG,[Never forget another riff idea. Works perfect...,Never forget another riff idea. Works perfectl...
B01HHZAU3W,"[Cute tuners. My daughters love it., The girlf...",Cute tuners. My daughters love it. The girlfri...
B01HI1VM28,"[great for guitar pot nuts, also makes a nice ...","great for guitar pot nuts, also makes a nice l..."


### Tf-Idf

In [6]:
def tokenizer(text):
    result = []
    for word in word_tokenize(text.lower()):
        if (re.search(r"[^a-z0-9 ]", word) is None) and word not in stopwords:
            result.append(word)
    return result

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
vec = TfidfVectorizer(tokenizer=tokenizer)

In [49]:
vec.fit(df_agg["texts"])

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function tokenizer at 0x7ffa17bbec10>)

In [50]:
voc = vec.get_feature_names_out()

In [51]:
top_10 = np.argpartition(vec.idf_, -100)[-100:]

In [52]:
voc[top_10]

array(['btr', 'modrl', 'btb455', 'btb1405e', 'modulesand', 'bt4s',
       'moecks', 'moer', 'btq', 'mocking', 'modi2', 'modmics', 'modmic',
       'modlines', 'modler', 'modle', 'moding', 'btr2000', 'btrak',
       'modifiy', 'modifiing', 'modifies', 'bttaaaaaannggg', 'bttm',
       'modifcations', 'modication', 'mocrophone', 'mofi', 'mk25',
       'mochipet', 'moccasins', 'moca', 'mobo', 'mobley', 'mobilpre',
       'mobilized', 'mobilesheets', 'buckos', 'mobilepod', 'mobilecube',
       'moba', 'mob', 'moar', 'moans', 'moaned', 'buckskin', 'mo9unt',
       'mo6', 'mnufacturer', 'mnr1', 'mnimos', 'mni', 'mnemonic', 'mn108',
       'mmx10', 'mmx', 'buddha', 'mms1', 'buddhist', 'mmmph',
       'mmmmmmmmmm', 'mmmmmmmmm', 'mmmmmmm', 'mmmmmmd', 'mmmmmm',
       'buddys', 'mmm12', 'budet', 'mmic', 'mmf', 'mmacss', 'mma', 'mm9',
       'mm50', 'mm160', 'mm1', 'budgetary', 'mlx990', 'mlw', 'mlthe',
       'mlt', 'mls', 'mlikely', 'mlg', 'mlcrophone', 'mlc', 'mlange',
       'mla', 'ml2', 'bud

Эти слова похожи на рандом. Возможно, обозначения типа "mks20" являются какими-то идентификаторами товаров. Как вариант, можно было бы отсортировать вектор и взять не самые большие значения idf, которые больше похожи на выбросы, а где-то по середине. Но я в итоге перешла к следующему варианту, который оказался лучше

### CountVectorizer

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vectorizer = CountVectorizer(tokenizer=tokenizer)

In [11]:
X = vectorizer.fit_transform(df_agg["texts"].tolist())

In [12]:
words = vectorizer.get_feature_names_out()

In [46]:
NE = []

for i in range(X.shape[0]):
    if X[i].max() > 10:
        top_10 = np.argpartition(X[i].toarray()[0], -5)[-5:]
        top_words = nltk.pos_tag(words[top_10])
        NE.extend([w for w, p in top_words if "NN" in p])

In [47]:
len(set(NE))

1525

In [57]:
NE = set(NE)

Здесь сущности больше похожи на NE

## 3.

In [56]:
df_agg["tokenized"] = df_agg["texts"].apply(tokenizer)

In [68]:
tokens = [item for sublist in df_agg["tokenized"].tolist() for item in sublist]

In [65]:
bigram_measures = nltk.collocations.BigramAssocMeasures()

In [78]:
finder = nltk.collocations.BigramCollocationFinder.from_words(tokens)

In [72]:
len(finder.score_ngrams(bigram_measures.likelihood_ratio))

2250677

In [79]:
finder.apply_word_filter(lambda x: x not in NE)

In [80]:
len(finder.score_ngrams(bigram_measures.likelihood_ratio))

206664

## 4.

In [84]:
finder.nbest(bigram_measures.likelihood_ratio, 30)

[('les', 'paul'),
 ('power', 'supply'),
 ('works', 'well'),
 ('ernie', 'ball'),
 ('planet', 'waves'),
 ('gig', 'bag'),
 ('easy', 'use'),
 ('phantom', 'power'),
 ('good', 'quality'),
 ('pedal', 'board'),
 ('acoustic', 'guitar'),
 ('work', 'well'),
 ('pop', 'filter'),
 ('easy', 'install'),
 ('home', 'studio'),
 ('shock', 'mount'),
 ('truss', 'rod'),
 ('strap', 'locks'),
 ('mic', 'stand'),
 ('sound', 'quality'),
 ('light', 'weight'),
 ('good', 'price'),
 ('price', 'range'),
 ('tube', 'amp'),
 ('well', 'worth'),
 ('e', 'string'),
 ('good', 'product'),
 ('bottom', 'line'),
 ('solid', 'state'),
 ('audio', 'interface')]

In [85]:
finder.nbest(bigram_measures.pmi, 30)

[('fiio', 'd3'),
 ('kevin', 'berry'),
 ('rocktron', 'hush'),
 ('oak', 'grigsby'),
 ('kaplan', 'amo'),
 ('m3', 'm3'),
 ('baroque', 'bassoon'),
 ('pig', 'hog'),
 ('zt', 'lunchbox'),
 ('kaplan', 'vivo'),
 ('scrambler', 'scrambler'),
 ('hd600', 'r70x'),
 ('denon', 'dl'),
 ('jet', 'falcon'),
 ('atm650', 'atm650'),
 ('iron', 'cobra'),
 ('egg', 'shakers'),
 ('vic', 'firth'),
 ('la', 'voz'),
 ('suction', 'cups'),
 ('novation', 'audiohub'),
 ('shoe', 'laces'),
 ('dbx', 'driverack'),
 ('lsr308', 'hs8'),
 ('mountain', 'dulcimer'),
 ('grosgrain', 'ribbon'),
 ('alignment', 'protractor'),
 ('hush', 'rocktron'),
 ('lee', 'oskar'),
 ('ou5', 'ou5')]

In [86]:
finder.nbest(bigram_measures.student_t, 30)

[('works', 'well'),
 ('good', 'quality'),
 ('easy', 'use'),
 ('acoustic', 'guitar'),
 ('les', 'paul'),
 ('work', 'well'),
 ('sound', 'quality'),
 ('good', 'price'),
 ('power', 'supply'),
 ('pedal', 'board'),
 ('gig', 'bag'),
 ('good', 'product'),
 ('sound', 'good'),
 ('mic', 'stand'),
 ('sounds', 'good'),
 ('ernie', 'ball'),
 ('well', 'worth'),
 ('phantom', 'power'),
 ('sound', 'like'),
 ('feel', 'like'),
 ('planet', 'waves'),
 ('easy', 'install'),
 ('price', 'range'),
 ('sounds', 'like'),
 ('looks', 'like'),
 ('home', 'studio'),
 ('tube', 'amp'),
 ('guitar', 'strap'),
 ('guitar', 'strings'),
 ('light', 'weight')]

Кажется, выбивается только pmi, выдает какие-то неинформативные коллокации. Оставшиеся два метода работают схоже, так что можно выбрать либо likelihood_ratio, либо student_t

## 5.

In [61]:
bgs = defaultdict(list)

for tokens in df_agg["tokenized"]:
    for i, token in enumerate(tokens):
        if token in NE:
            if i > 0:
               bgs[token].append([tokens[i-1], token])
            if i < len(tokens) - 2:
                bgs[token].append([token, tokens[i+1]])

Я вывела по 10 биграмм для 5 товаров, потому что биграмм слишком много

In [64]:
i = 0
for token in bgs:
    print(token)
    print("-----------------")
    for bg in bgs[token][:10]:
        print(" ".join(bg))
    print("\n\n")
    i += 1
    if i > 5:
        break

good
-----------------
good beginners
lots good
good instructions
pretty good
good book
really good
good instructor
values good
good views
app good



ukulele
-----------------
starter ukulele
ukulele kit
learn ukulele
ukulele received
firebrand ukulele
ukulele laminated
specialty ukulele
ukulele shop
larger ukulele
ukulele one



kit
-----------------
ukulele kit
kit everything
bought kit
kit towards
ukulele kit
kit got
ordered kit
kit price
starter kit
kit see



everything
-----------------
kit everything
everything need
luckily everything
everything inside
hear everything
everything levels
away everything
everything try
djx everything
everything perfect



need
-----------------
everything need
need learn
thing need
need either
going need
need lot
likely need
need acceptable
still need
need tuning



order
-----------------
according order
order history
chance order
order buy
playable order
order gain
rubbing order
order stop
friend order
order one



